# Usage

In [ ]:
# To run in in Google Colab, uncomment the following:

# !pip install expertsystem graphviz

## Main interface

The {mod}`expertsystem` consists of three main components: {mod}`~.particle`, {mod}`~.reaction`, and {mod}`~.amplitude` that build on each other. Here's a small example of how to use them!

In [ ]:
import expertsystem as es

pdg = es.particle.load_pdg()
pdg.filter(lambda p: p.spin == 2 and p.charmness).names

In [ ]:
result = es.reaction.generate(
    initial_state="J/psi(1S)",
    final_state=["K0", "Sigma+", "p~"],
    allowed_interaction_types="strong",
    formalism_type="canonical-helicity",
)

In [ ]:
import graphviz

dot = es.io.asdot(result, collapse_graphs=True)
graphviz.Source(dot)

Finally, you use the {mod}`.amplitude` module to convert these transitions into a mathematical description that you can use to fit your data and perform {doc}`Partial Wave Analysis <pwa:index>`!

## Advanced

Each of the {mod}`expertsystem`'s sub-modules offer functionality to handle more advanced reaction types. The following notebooks illustrate how use them.

```{toctree}
---
maxdepth: 2
---
usage/reaction
usage/amplitude
usage/particle
usage/visualize
usage/dynamics
```